# OAuth

- Twitter uses an [OAuth 1.0a for user and application-only authentication](https://dev.twitter.com/oauth). Brittle Wit does not support application-only authentication. (It seemed like extra code complexity for an edge case easily-solved by using the application's twitter user as a user.) 
- It uses a nonce of 42 characters sampled over numbers and sensitive letters. This leads to $42^{62}$ possibilities which should be "relatively random."

## Sample Authentication with `requests`

In [18]:
from requests import request as _request
from brittle_wit_core import (AppCredentials,
                              obtain_request_token, 
                              extract_access_token, 
                              redirect_url, 
                              obtain_access_token,
                              extract_request_token)

def obfuscate(s):
    return s[0] + "*"*(len(s)-1) if isinstance(s, str) else -1


# Loads via TWITTER_APP_KEY, TWITTER_APP_SECRET environmental variables.
APP_CRED = AppCredentials.load_from_env()

In [19]:
twitter_req, headers = obtain_request_token(APP_CRED)

In [25]:
resp = _request(twitter_req.method, 
                twitter_req.url,             
                params=twitter_req.params, 
                headers=headers)

oauth_token, oauth_secret = extract_request_token(resp.status_code, resp.content.decode('utf8'))

In [26]:
url = redirect_url(oauth_token)
!open $url

In [41]:
PIN_FROM_BROWSER = '3230924'

twitter_req, headers = obtain_access_token(APP_CRED, oauth_token, PIN_FROM_BROWSER)

resp = _request(twitter_req.method, 
                twitter_req.url,             
                params=twitter_req.params, 
                headers=headers)
d = extract_access_token(resp.status_code, resp.content.decode('utf8'))

d = {k: obfuscate(v) for k, v in d.items()}
d

{'oauth_token': '2*************************************************',
 'oauth_token_secret': 'q********************************************',
 'screen_name': 'p*******',
 'user_id': -1,
 'x_auth_expires': '0'}